# Machine Learning Earthquake Alert Prediction

## Methods

### Data Preprocessing

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import PowerTransformer, StandardScaler, PolynomialFeatures, LabelEncoder, label_binarize
from sklearn.metrics import ConfusionMatrixDisplay, classification_report, confusion_matrix, f1_score, balanced_accuracy_score, roc_auc_score, brier_score_loss, roc_curve, auc, precision_recall_curve, average_precision_score
from xgboost import XGBClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler
from pathlib import Path

In [ ]:
for p in ["../data/sample/dataset_sample.csv",
          "../data/raw/earthquake_alert_balanced_dataset.csv"]:
    if Path(p).exists():
        df = pd.read_csv(p)
        break
else:
    raise FileNotFoundError("Place sample in data/sample/ or full file in data/raw/")

print(df.shape)
df.head()

In [ ]:
df.info()
df.describe()
df.isnull().sum()

In [ ]:
X = df[['magnitude', 'depth', 'cdi', 'mmi', 'sig']]
y = df['alert']

In [ ]:
pt = PowerTransformer()
df[['cdi', 'mmi']] = pt.fit_transform(df[['cdi', 'mmi']])

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 4))
sns.histplot(df['cdi'], ax=axes[0], kde=True)
axes[0].set_title('CDI after PowerTransform')
sns.histplot(df['mmi'], ax=axes[1], kde=True)
axes[1].set_title('MMI after PowerTransform')
plt.show()

In [ ]:
df_original = pd.read_csv("../data/earthquake_alert_balanced_dataset.csv")

fig, axes = plt.subplots(2, 2, figsize=(10, 8))

sns.histplot(df_original['cdi'], ax=axes[0, 0], kde=True)
axes[0, 0].set_title('CDI Before PowerTransform')

sns.histplot(df_original['mmi'], ax=axes[0, 1], kde=True)
axes[0, 1].set_title('MMI Before PowerTransform')

sns.histplot(df['cdi'], ax=axes[1, 0], kde=True)
axes[1, 0].set_title('CDI After PowerTransform')

sns.histplot(df['mmi'], ax=axes[1, 1], kde=True)
axes[1, 1].set_title('MMI After PowerTransform')

plt.tight_layout()
plt.show()

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
pd.DataFrame(X_scaled, columns=X.columns).head()

In [ ]:
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_scaled, y)

print("Before oversampling:\n", y.value_counts(), "\n")
print("After oversampling:\n", pd.Series(y_resampled).value_counts())

### Models

#### Logistic Regression

In [ ]:
# Training/Testing split
df = pd.read_csv("../data/earthquake_alert_balanced_dataset.csv")
N = df[['magnitude', 'depth', 'cdi', 'mmi', 'sig']]
D = df['alert']

NTrain, NTest, DTrain, DTest = train_test_split(
    N, D, test_size=0.20, stratify=D, random_state=42
)

# Building the Pipeline
features = ['magnitude', 'depth', 'cdi', 'mmi', 'sig']

preprocess = ColumnTransformer(
    transformers=[
        ("power", PowerTransformer(), ['cdi', 'mmi']),
        ("pass", 'passthrough', features),
    ],
    remainder='drop',
)

pipeline = Pipeline(steps=[
    ("preprocess", preprocess),
    ("poly", "passthrough"),
    ("scale", StandardScaler()),
    ("sampler", "passthrough"),
    ("model", LogisticRegression(solver="lbfgs", max_iter=1000)),
])

# Training and Evaluating
parameters = [{
    "poly": ["passthrough", PolynomialFeatures(degree=2, include_bias=False)],
    "sampler": ["passthrough", RandomOverSampler(random_state=42)],
    "model__C": [0.25, 0.5, 1.0, 2.0, 4.0],
    "model__class_weight": [None, "balanced"],
}]

cross = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid = GridSearchCV(
    estimator=pipeline,
    param_grid=parameters,
    scoring="f1_macro",
    cv=cross,
    n_jobs=-1,
    verbose=1,
)

grid.fit(NTrain, DTrain)
best_model = grid.best_estimator_

y_pred = best_model.predict(NTest)

print("\nClassification report:")
print(classification_report(DTest, y_pred, digits=3))

#### Random Forest

In [ ]:
df = pd.read_csv("../data/earthquake_alert_balanced_dataset.csv")
X = df[['magnitude', 'depth', 'cdi', 'mmi', 'sig']].copy()
y = df['alert']

# Power transform CDI/MMI, then scale all features
pt = PowerTransformer()
X[['cdi', 'mmi']] = pt.fit_transform(X[['cdi', 'mmi']])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, stratify=y, random_state=42
)

rf = RandomForestClassifier(random_state=42)

params = {
    "n_estimators": [100, 200, 300],
    "max_depth": [None, 5, 10, 20],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 2],
    "class_weight": [None, "balanced"],
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_rf = GridSearchCV(
    rf,
    params,
    scoring="f1_macro",
    cv=cv,
    n_jobs=-1,
    verbose=1,
)

grid_rf.fit(X_train, y_train)
best_rf = grid_rf.best_estimator_

y_pred_rf = best_rf.predict(X_test)

print("Best params:", grid_rf.best_params_)
print("\nClassification Report:\n", classification_report(y_test, y_pred_rf, digits=3))

#### Gradient Boosting (XGBoost)

In [ ]:
# Encode string labels into integers for XGBoost
le = LabelEncoder()
DTrain_enc = le.fit_transform(DTrain)
DTest_enc = le.transform(DTest)

features = ['magnitude', 'depth', 'cdi', 'mmi', 'sig']

preprocess_gb = ColumnTransformer(
    transformers=[
        ("power", PowerTransformer(), ['cdi', 'mmi']),
        ("pass", "passthrough", features),
    ],
    remainder='drop',
)

gb_model = Pipeline(steps=[
    ("preprocess", preprocess_gb),
    ("poly", PolynomialFeatures(degree=2, include_bias=False)),
    ("scale", StandardScaler()),
    ("sampler", RandomOverSampler(random_state=42)),
    ("model", XGBClassifier(
        objective="multi:softprob",
        num_class=4,
        eval_metric="mlogloss",
        tree_method="hist",
        random_state=42,
        n_estimators=200,
        max_depth=4,
        learning_rate=0.1,
        subsample=0.9,
        colsample_bytree=0.9,
        n_jobs=1,
    )),
])

gb_model.fit(NTrain, DTrain_enc)

y_pred_enc = gb_model.predict(NTest)
y_prob_gb = gb_model.predict_proba(NTest)

classes_gb = le.classes_
y_pred_gb = le.inverse_transform(y_pred_enc)

print("Classification report:")
print(classification_report(DTest, y_pred_gb, target_names=classes_gb, digits=3))

## Results

### Logistic Regression

#### Quantitative Metrics

In [ ]:
# Predictions & probabilities
y_pred = best_model.predict(NTest)
y_prob = best_model.predict_proba(NTest)
classes = best_model.classes_
print("Classes:", list(classes))

# Compute quantitative metrics
macro_f1 = f1_score(DTest, y_pred, average='macro')
balanced_acc = balanced_accuracy_score(DTest, y_pred)

Y_true_ohe = pd.get_dummies(DTest).reindex(columns=classes, fill_value=0)
macro_roc_auc = roc_auc_score(
    Y_true_ohe, y_prob, multi_class='ovr', average='macro'
)

brier_macro = np.mean([
    brier_score_loss((np.array(DTest) == c).astype(int), y_prob[:, i])
    for i, c in enumerate(classes)
])

print(f"Macro F1: {macro_f1:.3f}")
print(f"Balanced Accuracy: {balanced_acc:.3f}")
print(f"Macro ROC-AUC: {macro_roc_auc:.3f}")
print(f"Macro Brier (↓): {brier_macro:.3f}")

# Per-class performance table
report = classification_report(
    DTest, y_pred, target_names=classes, output_dict=True
)
pd.DataFrame(report).T

#### Visualizations

In [ ]:
# Confusion Matrix (Counts)
cm = confusion_matrix(DTest, y_pred, labels=classes)
fig, ax = plt.subplots(figsize=(6, 6))
ConfusionMatrixDisplay(cm, display_labels=classes).plot(
    ax=ax, cmap="Blues", colorbar=False
)
ax.set_title("Confusion Matrix - Logistic Regression")
plt.tight_layout()
plt.show()

# Confusion Matrix (Normalized by row / recall)
cm_norm = confusion_matrix(DTest, y_pred, labels=classes, normalize="true")
fig, ax = plt.subplots(figsize=(6, 6))
ConfusionMatrixDisplay(cm_norm, display_labels=classes).plot(
    ax=ax, cmap="Blues", colorbar=True
)
ax.set_title("Normalized Confusion Matrix (per-class recall)")
plt.tight_layout()
plt.show()

In [ ]:
# ROC curves (One-vs-Rest)
Y_true_bin = label_binarize(DTest, classes=classes)

plt.figure(figsize=(7, 6))
for i, c in enumerate(classes):
    fpr, tpr, _ = roc_curve(Y_true_bin[:, i], y_prob[:, i])
    auc_i = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"{c} (AUC = {auc_i:.2f})")

plt.plot([0, 1], [0, 1], 'k--')
plt.title("ROC Curves - Logistic Regression (One-vs-Rest)")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Precision-Recall curves
plt.figure(figsize=(7, 6))
for i, c in enumerate(classes):
    precision, recall, _ = precision_recall_curve(Y_true_bin[:, i], y_prob[:, i])
    ap = average_precision_score(Y_true_bin[:, i], y_prob[:, i])
    plt.plot(recall, precision, label=f"{c} (AP = {ap:.2f})")

plt.title("Precision-Recall Curves - Logistic Regression")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend()
plt.tight_layout()
plt.show()

### Random Forest

#### Quantitative Metrics

In [ ]:
# Probabilities for metrics/curves
y_prob_rf = best_rf.predict_proba(X_test)
classes_rf = best_rf.classes_
print("Classes:", list(classes_rf))

macro_f1_rf = f1_score(y_test, y_pred_rf, average='macro')
balanced_acc_rf = balanced_accuracy_score(y_test, y_pred_rf)

Y_true_ohe_rf = pd.get_dummies(y_test).reindex(columns=classes_rf, fill_value=0)
macro_roc_auc_rf = roc_auc_score(
    Y_true_ohe_rf, y_prob_rf, multi_class='ovr', average='macro'
)

brier_macro_rf = np.mean([
    brier_score_loss((np.array(y_test) == c).astype(int), y_prob_rf[:, i])
    for i, c in enumerate(classes_rf)
])

print(f"Macro F1: {macro_f1_rf:.3f}")
print(f"Balanced Accuracy: {balanced_acc_rf:.3f}")
print(f"Macro ROC-AUC: {macro_roc_auc_rf:.3f}")
print(f"Macro Brier (↓): {brier_macro_rf:.3f}")

report_rf = classification_report(
    y_test, y_pred_rf, target_names=classes_rf, output_dict=True
)
pd.DataFrame(report_rf).T

#### Visualizations

In [ ]:
# Confusion Matrix (Counts)
labels_rf = classes_rf  # or sorted(y.unique())
cm_rf = confusion_matrix(y_test, y_pred_rf, labels=labels_rf)
fig, ax = plt.subplots(figsize=(6, 6))
ConfusionMatrixDisplay(cm_rf, display_labels=labels_rf).plot(
    ax=ax, cmap="Blues", colorbar=False
)
ax.set_title("Random Forest Confusion Matrix")
plt.tight_layout()
plt.show()

# Confusion Matrix (Normalized)
cm_norm_rf = confusion_matrix(
    y_test, y_pred_rf, labels=labels_rf, normalize="true"
)
fig, ax = plt.subplots(figsize=(6, 6))
ConfusionMatrixDisplay(cm_norm_rf, display_labels=labels_rf).plot(
    ax=ax, cmap="Blues", colorbar=True
)
ax.set_title("Normalized Confusion Matrix - Random Forest")
plt.tight_layout()
plt.show()

In [ ]:
# ROC Curves (One-vs-Rest)
Y_bin_rf = label_binarize(y_test, classes=labels_rf)

plt.figure(figsize=(7, 6))
for i, c in enumerate(labels_rf):
    fpr, tpr, _ = roc_curve(Y_bin_rf[:, i], y_prob_rf[:, i])
    auc_i = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"{c} (AUC = {auc_i:.2f})")

plt.plot([0, 1], [0, 1], 'k--')
plt.title("ROC Curves - Random Forest (One-vs-Rest)")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Precision-Recall Curves
plt.figure(figsize=(7, 6))
for i, c in enumerate(labels_rf):
    precision, recall, _ = precision_recall_curve(Y_bin_rf[:, i], y_prob_rf[:, i])
    ap = average_precision_score(Y_bin_rf[:, i], y_prob_rf[:, i])
    plt.plot(recall, precision, label=f"{c} (AP = {ap:.2f})")

plt.title("Precision-Recall Curves - Random Forest")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend()
plt.tight_layout()
plt.show()

### Gradient Boosting (XGBoost)

#### Quantitative Metrics

In [ ]:
# Macro metrics
macro_f1_gb = f1_score(DTest, y_pred_gb, average='macro')
balanced_acc_gb = balanced_accuracy_score(DTest, y_pred_gb)

Y_true_ohe_gb = pd.get_dummies(DTest).reindex(columns=classes_gb, fill_value=0)
macro_roc_auc_gb = roc_auc_score(
    Y_true_ohe_gb, y_prob_gb, multi_class='ovr', average='macro'
)

macro_brier_gb = np.mean([
    brier_score_loss((np.array(DTest) == c).astype(int), y_prob_gb[:, i])
    for i, c in enumerate(classes_gb)
])

print(f"Macro F1: {macro_f1_gb:.3f}")
print(f"Balanced Accuracy: {balanced_acc_gb:.3f}")
print(f"Macro ROC-AUC: {macro_roc_auc_gb:.3f}")
print(f"Macro Brier (↓): {macro_brier_gb:.3f}")

# Per-class performance table
report_gb = classification_report(
    DTest, y_pred_gb, target_names=classes_gb, output_dict=True
)
pd.DataFrame(report_gb).T

#### Visualizations

In [ ]:
# Confusion Matrix (Counts)
cm_gb = confusion_matrix(DTest, y_pred_gb, labels=classes_gb)
fig, ax = plt.subplots(figsize=(6, 6))
ConfusionMatrixDisplay(cm_gb, display_labels=classes_gb).plot(
    ax=ax, cmap="Blues", colorbar=False, values_format='d'
)
ax.set_title("Confusion Matrix - Gradient Boosting (XGBoost)")
plt.tight_layout()
plt.show()

# Confusion Matrix (Normalized)
cm_norm_gb = confusion_matrix(
    DTest, y_pred_gb, labels=classes_gb, normalize="true"
)
fig, ax = plt.subplots(figsize=(6, 6))
ConfusionMatrixDisplay(cm_norm_gb, display_labels=classes_gb).plot(
    ax=ax, cmap="Blues", colorbar=True
)
ax.set_title("Normalized Confusion Matrix - Gradient Boosting")
plt.tight_layout()
plt.show()

In [ ]:
# ROC Curves (One-vs-Rest)
Y_bin_gb = label_binarize(DTest, classes=classes_gb)

plt.figure(figsize=(7, 6))
for i, c in enumerate(classes_gb):
    fpr, tpr, _ = roc_curve(Y_bin_gb[:, i], y_prob_gb[:, i])
    auc_i = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"{c} (AUC = {auc_i:.2f})")

plt.plot([0, 1], [0, 1], 'k--')
plt.title("ROC Curves - Gradient Boosting (One-vs-Rest)")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Precision-Recall Curves
plt.figure(figsize=(7, 6))
for i, c in enumerate(classes_gb):
    precision, recall, _ = precision_recall_curve(Y_bin_gb[:, i], y_prob_gb[:, i])
    ap = average_precision_score(Y_bin_gb[:, i], y_prob_gb[:, i])
    plt.plot(recall, precision, label=f"{c} (AP = {ap:.2f})")

plt.title("Precision-Recall Curves - Gradient Boosting")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend()
plt.tight_layout()
plt.show()